In [38]:
from exprcol import register, resolve, REGISTRY

import numpy as np
import operator
import pandas as pd
from dask import dataframe as ddf

In [8]:
@register('discrim', ['a', 'b', 'c'])
def discriminant(a, b, c):
    return b**2 - 4*a*c

@register('pos_num', ['b', 'sqrt_discrim'])
def pos_num(b, sqrt_discrim):
    return -b + sqrt_discrim

@register('neg_num', ['b', 'sqrt_discrim'])
def neg_num(b, sqrt_discrim):
    return -b - sqrt_discrim

register('double_a', ['a', 2])(operator.mul)
register('pos_root', ['pos_num', 'double_a'])(operator.truediv)
register('neg_root', ['neg_num', 'double_a'])(operator.truediv)
register('sqrt_discrim', ['discrim', .5])(pow)

<function pow(x, y, z=None, /)>

In [62]:
data = pd.DataFrame({'a': np.random.rand(10000), 'b': 4*np.random.rand(10000), 'c': np.random.rand(10000)})

In [63]:
data

,a,b,c
0,0.087683,3.733088,0.791449
1,0.244657,0.999731,0.287873
2,0.238428,2.990638,0.243787
3,0.917214,0.759941,0.771665
4,0.030690,3.371331,0.150902
...,...,...,...
9995,0.482493,1.342553,0.620086
9996,0.482065,2.845487,0.171563
9997,0.316553,3.997872,0.497792
9998,0.509177,0.924700,0.970953


In [73]:
%%timeit
pos_root = (-data.b + (data.b**2 - 4*data.a*data.c) ** .5)/(2*data.a)
neg_root = (-data.b - (data.b**2 - 4*data.a*data.c) ** .5)/(2*data.a)

2.53 ms ± 228 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [74]:
%%timeit
roots = resolve.compute(data, ['pos_root', 'neg_root'])

1.79 ms ± 64.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
